### Multi Agent Group Task for Reasearch Paper Analysis and Summarization

Here, multi agent LLM system is designed using autogen's Agents and GroupChat. 

The implementation used is referred from [pdichone](https://github.com/pdichone)

#### Importing required libraries

In [7]:
import os
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

#### Configuring the LLM

Here, the configuration details are added as a list of dictionary. Open source model Llama is used here through Ollama.

In [8]:

config_list = [
    {
        "model": "llama3.1:8b",  # add your own model here
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
    },
]

llm_config = {"config_list": config_list, "temperature": 0.0}

#### Creating the Analysis System

This creates the system use for analysis by using AssistantAgent and UserProxyAgent from autogen. This sets up multiple agents where each agent needs a sustem message to indicate what kind of task is expected from the agent.

Once the agents are created, a group chat instance is setup which facilitates the group interaction between multipe agents. All the created agents are included in the group chat instance.

Group chat requires a manager and the manager is set up using GroupChatManager instance. The analysis task is divided into subtasks and each task is started by the group chat manager which results in all the agents working together to achieve each subtask.


In [9]:
class ResearchAnalysisSystem:
    def __init__(self):
        self.llm_config = llm_config

        self.code_execution_config = {
            "work_dir": "research_output",
            "use_docker": False,
        }

        self.setup_agents()
        self.setup_group_chat()

    def setup_agents(self):
        # User Proxy for coordination
        self.user_proxy = UserProxyAgent(
            name="admin",
            human_input_mode="NEVER",
            code_execution_config=self.code_execution_config,
            system_message="Admin coordinating research analysis.",
        )

        # Specialized Agents
        self.research_agent = AssistantAgent(
            name="researcher",
            llm_config=self.llm_config,
            system_message="""Find and analyze research papers. Focus on:
            1. Identifying relevant papers
            2. Extracting key findings
            3. Categorizing applications""",
        )

        self.data_analyst = AssistantAgent(
            name="analyst",
            llm_config=self.llm_config,
            system_message="""Analyze research data to:
            1. Extract metrics
            2. Generate visualizations
            3. Identify trends""",
        )

        self.reviewer = AssistantAgent(
            name="reviewer",
            llm_config=self.llm_config,
            system_message="""Review findings for:
            1. Accuracy
            2. Completeness
            3. Relevance""",
        )

        self.content_writer = AssistantAgent(
            name="writer",
            llm_config=self.llm_config,
            system_message="""Synthesize findings into clear reports with:
            1. Executive summaries
            2. Detailed analysis
            3. Recommendations""",
        )

    def setup_group_chat(self):
        # Create agent group
        self.agents = [
            self.user_proxy,
            self.research_agent,
            self.data_analyst,
            self.reviewer,
            self.content_writer,
        ]

        # Initialize group chat
        self.group_chat = GroupChat(agents=self.agents, messages=[], max_round=50)

        # Set up manager
        self.manager = GroupChatManager(
            groupchat=self.group_chat, llm_config=self.llm_config
        )

    def research_pipeline(self, topic):
        # Initialize research tasks
        tasks = [
            f"Research papers on {topic}",
            "Extract and categorize applications",
            "Generate visualization of findings",
            "Prepare comprehensive report",
        ]

        results = []
        for task in tasks:
            # Each task is processed by the group
            result = self.user_proxy.initiate_chat(
                self.manager, message=task, summary_method="last_msg"
            )
            results.append(result)

        return results

    def analyze_article(self, article_content):
        # Article analysis pipeline
        analysis_tasks = [
            f"Analyze structure and coherence: {article_content}",
            "Review style and tone",
            "Verify factual accuracy",
            "Provide improvement suggestions",
            "Generate final publication readiness report",
        ]

        analysis_results = []
        for task in analysis_tasks:
            result = self.user_proxy.initiate_chat(
                self.manager, message=task, summary_method="last_msg"
            )
            analysis_results.append(result)

        return analysis_results




#### Invoking the system

When supplied with topic, the invoked ResearchAnalysisSystem will apply all the subtasks from the tasks list. Also, a locak file 'article.txt' is fed into 'analyze_article' method which will do all the analysis subtasks and collect the results.

In [10]:
def main():
    # Initialize system
    system = ResearchAnalysisSystem()

    # Example usage
    topic = "machine learning in healthcare"
    research_results = system.research_pipeline(topic)

    # Article analysis
    with open("data/article.txt", "r") as file:
        article_content = file.read()

    analysis_results = system.analyze_article(article_content)

    # Print final results
    print("\nResearch Analysis Results:")
    for i, result in enumerate(research_results):
        print(f"\nTask {i+1} Results:")
        print(result)

    print("\nArticle Analysis Results:")
    for i, result in enumerate(analysis_results):
        print(f"\nAnalysis Task {i+1} Results:")
        print(result)


if __name__ == "__main__":
    main()

admin (to chat_manager):

Research papers on machine learning in healthcare

--------------------------------------------------------------------------------
[autogen.oai.client: 12-12 00:17:22] {732} WARNING - Model llama3.1:8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 12-12 00:17:23] {732} WARNING - Model llama3.1:8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: researcher

[autogen.oai.client: 12-12 00:17:48] {732} WARNING - Model llama3.1:8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
researcher (to chat_manager):

Here are some research papers on machine learning in healthcare:

**Paper 1:**

* **Title:** "Deep Learning f

KeyboardInterrupt: 